In [4]:
from pathlib import Path
from tqdm import tqdm
from virtualizarr import open_virtual_dataset, open_virtual_mfdataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
from icechunk.xarray import to_icechunk
from dask.diagnostics import ProgressBar

import obstore as obs
import xarray as xr
import pandas as pd
import icechunk
import hvplot.xarray

import warnings
warnings.filterwarnings(
  "ignore",
  message="Numcodecs codecs are not in the Zarr version 3 specification*",
  category=UserWarning
)
icechunk.set_logs_filter(
    "icechunk::storage::object_store=error"
)  # remove the warning due to the local icechunk store

In [2]:
ROOT_DIR = Path("/Volumes/T7")
RAW_FILES_DIR = ROOT_DIR / "raw_files/hackathon-meteo-france"

BUCKETS_DIR = ROOT_DIR / "buckets"
ICECHUNK_BUCKET = BUCKETS_DIR / "hackathon-meteo-france"
PREFIX_ANASTASIA = "REA/ANASTASIA"

prefix = "file:///Volumes/"

In [8]:
df = pd.read_csv(RAW_FILES_DIR / "SAFRAN" / "QUOT_SIM2_1958-1959.csv", delimiter=";")

In [9]:
df.head()

,LAMBX,LAMBY,DATE,PRENEI,PRELIQ,T,FF,Q,DLI,SSI,...,RESR_NEIGE6,HTEURNEIGE,HTEURNEIGE6,HTEURNEIGEX,SNOW_FRAC,ECOULEMENT,WG_RACINE,WGI_RACINE,TINF_H,TSUP_H
0,600,24010,19580801,0.0,0.0,15.3,3.5,8.669,2710.7,1280.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.223,0.0,11.9,17.8
1,600,24010,19580802,0.0,0.0,14.0,3.1,7.751,3090.1,1449.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.223,0.0,10.3,17.6
2,600,24010,19580803,0.0,0.0,15.0,3.9,8.956,3491.8,1045.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.223,0.0,11.2,19.1
3,600,24010,19580804,0.0,0.0,16.4,4.1,9.825,3556.7,1350.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.222,0.0,14.7,17.8
4,600,24010,19580805,0.0,9.2,16.8,4.1,10.938,3553.3,782.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.222,0.0,16.6,17.4


In [10]:
ds = xr.Dataset.from_dataframe(df)

In [11]:
ds

<xarray.Dataset> Size: 1GB
Dimensions:      (index: 5124056)
Coordinates:
  * index        (index) int64 41MB 0 1 2 3 ... 5124052 5124053 5124054 5124055
Data variables: (12/29)
    LAMBX        (index) int64 41MB 600 600 600 600 ... 11960 11960 11960 11960
    LAMBY        (index) int64 41MB 24010 24010 24010 ... 17450 17450 17450
    DATE         (index) int64 41MB 19580801 19580802 ... 19591230 19591231
    PRENEI       (index) float64 41MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    PRELIQ       (index) float64 41MB 0.0 0.0 0.0 0.0 9.2 ... 0.0 0.0 0.0 0.0
    T            (index) float64 41MB 15.3 14.0 15.0 16.4 ... 11.1 10.3 15.7
    ...           ...
    SNOW_FRAC    (index) float64 41MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ECOULEMENT   (index) float64 41MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WG_RACINE    (index) float64 41MB 0.223 0.223 0.223 ... 0.33 0.328 0.327
    WGI_RACINE   (index) float64 41MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TINF_H       (index) float64 41MB 11.9 10.3 11.2 14.7 ... 12.7 7.7 4.8 8.8
    TSUP_H       (index) float64 41MB 17.8 17.6 19.1 17.8 ... 17.0 16.9 20.1